In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('2025_LoL_esports_match_data_from_OraclesElixir.csv')
print(f"Shape: {data.shape}")


Shape: (82908, 164)


C:\Users\Asus\AppData\Local\Temp\ipykernel_13232\2730736923.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2025_LoL_esports_match_data_from_OraclesElixir.csv')


In [ ]:
# Escrevendo base principal
# 